<a href="https://colab.research.google.com/github/ffreemt/app1/blob/master/succ_ycco_make_upload_model_s_MODEL_ID1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title pip
!pip install -q sentence-transformers huggingface_hub set_env_colab_kaggle_dotenv

In [ ]:
# @title login to hf
from huggingface_hub import login
from set_env import set_env

HF_TOKEN = set_env("HF_TOKEN")
login(HF_TOKEN)

In [ ]:
# @title create and joblib.dump sentence-transformers/distiluse-base-multilingual-cased-v1

from pathlib import Path
import joblib

from sentence_transformers import SentenceTransformer

model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"  # .539GB

%time model = SentenceTransformer(model_name)  # 10s 2.44s 1.61s

# if not Path("model-dir").exist():
Path("model_dir").mkdir(parents=True, exist_ok=True)

%time joblib.dump(model, "model_dir/model-s-v2")

In [ ]:
# @title create and upload hf REPO_ID
REPO_ID = 'mikeee/model_s_512v2' # @param {type:"string"}
from huggingface_hub import HfApi
api = HfApi()

api.create_repo(REPO_ID, exist_ok=True)

api.upload_folder(
    folder_path="model_dir",
    repo_id=REPO_ID,
    repo_type="model",
)

In [ ]:
!ls -l /content/model_dir

In [ ]:
url = hf_hub_url("mikeee/model_s_512", "model-s")  # mikeee/model_s_512

# https://huggingface.co/mikeee/model_s_512/resolve/main/model_s
# https://huggingface.co/mikeee/model_s_512/resolve/main/model-s

url

In [ ]:
!apt update -qq && apt install aria2 -q

In [ ]:
!rm model-s
# !wget -c {url}

In [ ]:
filename = Path(url).name
path = "."
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url} -d {path} -o {filename}

In [ ]:
hf_hub_download?

In [ ]:
import joblib
from huggingface_hub import hf_hub_url, hf_hub_download

%time loc = hf_hub_download('mikeee/model_s_512', 'model-s', local_dir="local_dir")
# 21.5 s, 231 ms

display(f"{loc=}")
%time model = joblib.load(loc)  # 1.78

In [ ]:
# model.encode?
# batch_size: int = 32,
# convert_to_numpy: bool = True
# show_progress_bar: bool = None
# normalize_embeddings: bool = False
# -> numpy.ndarray

In [ ]:
# @title loadtext en zh
try:
  import loadtext
except ModuleNotFoundError:
  !pip install -q loadtext

from loadtext import loadtext

!git clone https://github.com/ffreemt/en-de-zh-txt  # https://github.com/qdrant/fastembed

en = loadtext("en-de-zh-txt/fangfang-en.txt", True)
zh = loadtext("en-de-zh-txt/fangfang-zh.txt", True)

test_en = loadtext("en-de-zh-txt/test_en.txt", 1)
test_zh = loadtext("en-de-zh-txt/test_zh.txt", 1)

display([len(en), len(zh), len(test_en), len(test_zh)])

In [ ]:
ls en-de-zh-txt


In [ ]:
# @title  import SentenceTransformer
try:
  import sentence_transformers
except ModuleNotFoundError:
  !pip install -q sentence_transformers
from sentence_transformers import SentenceTransformer

MODEL_ID = "sentence-transformers/distiluse-base-multilingual-cased-v1"  # .539GB   # 10s 2.44s 1.61s
MODEL_ID = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"  # .47G 13.4 s

%time model = SentenceTransformer(MODEL_ID)


In [ ]:
%%time
vec_en = model.encode(
  en,
  batch_size=1,  # en[320] bs1 1min 8s, bs2 1min 8s, bs4 54.2 s
  # batch_size=10,  # en[320] bs8 53.9 53.8 s, bs10 52.4 s
  # batch_size=16,  # en[:320] bs16 53s 55.8 s
  # batch_size=64,   # en[:64], 15s, batch_size=32(default) 18s, en[:320] bs:32 60s 3.5s/it, bs64 1min4s
  show_progress_bar=True,
  normalize_embeddings=True,
)

# distiluse-base-multilingual-cased-v1
# en[:500] 1 min 30 s
# en bs32(default) 2min 37s 2min 35s, bs10 2min 29s, bs100 2min 57s, bs1 3min 35s

# sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
# en bs32(default) 1min 37s


In [ ]:
%%time
vec_zh = model.encode(
  zh,
  # batch_size=4,  #
  # batch_size=1,  #
  # batch_size=16,  #
  # batch_size=64,   #
  show_progress_bar=True,
  normalize_embeddings=True,
)

# distiluse-base-multilingual-cased-v1
# zh bs32(default) 2min 1s 2min 13s, bs10 2min 11s, bs1 745  3min 1s

# sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
# zh bs32(default) 1min

In [ ]:
# @title seaborn.heatmap
import seaborn as sns

# ax = sns.heatmap(vec_en.dot(vec_zh.T), cmap="Blues", cbar=True)

In [ ]:
# sns.heatmap(cmat.T, cmap="Blues", cbar=True)

In [ ]:
# sns.heatmap?

In [ ]:
import pandas as pd
df = pd.DataFrame(vec_en.dot(vec_zh.T))

df.index, df.columns

In [ ]:
df_index = df.index
df_columns = df.columns
df_index.rename("en", inplace=True)
df_columns.rename("zh", inplace=True)

display(df_index, df_columns)
df.index, df.columns = df_index, df_columns
df

In [ ]:
ax = sns.heatmap(df, cmap="Blues", cbar=True)

In [ ]:
loc = hf_hub_download("mikeee/model_s_512", "model-s", local_dir=".")

In [ ]:
"""
Load model_s from hf.

cf aslo align-model-pool\model_pool\load_model.py and ycco make-upload-model-s.ipynb.
"""
import joblib
from huggingface_hub import hf_hub_download
from loguru import logger

try:
    loc = hf_hub_download("mikeee/model_s_512", "model-s", local_dir=".")
except Exception as exc:
    logger.error(exc)
    raise SystemExit(1) from exc

def load_model_s(model_file=None):
    """Load a model from hf."""
    if model_file is None:
        model_file = loc
    try:
        model = joblib.load(model_file)
    except Exception as exc:
        logger.error(exc)
        raise
    return model


In [ ]:
%time model_s0 = load_model_s()

In [ ]:
all(model_s0.encode("test 测试") == model.encode("test 测试"))

In [ ]:
!ls -l en-de-zh-txt

In [ ]:
import numpy as np
paras1 = loadtext("en-de-zh-txt/sternstunden04-en.txt", splitlines=True)
paras2 = loadtext("en-de-zh-txt/sternstunden04-de.txt", True)

# red-sorghum_ch1_en.txt
paras1 = loadtext("en-de-zh-txt/red-sorghum_ch1_en.txt", splitlines=True)
paras2 = loadtext("en-de-zh-txt/red-sorghum_ch1_zh.txt", True)

paras1 = loadtext("en-de-zh-txt/fangfang-en.txt", splitlines=True)
paras2 = loadtext("en-de-zh-txt/fangfang-zh.txt", True)

display(len(paras1), len(paras2))

# cmat = gen_cmat(paras1, paras2)
cmat = np.dot(model.encode(paras2, show_progress_bar=1, normalize_embeddings=1), model.encode(paras1, show_progress_bar=1, normalize_embeddings=1).T)

sns.heatmap(cmat)

In [ ]:
sns.heatmap(cmat.T, cmap="Blues", cbar=True)

In [ ]:
MODEL_ID = "sentence-transformers/multi-qa-mpnet-base-dot-v1"  # .43G 8s dim768
%time model = SentenceTransformer(MODEL_ID)
%time cmat = np.dot(model.encode(paras2, show_progress_bar=1, normalize_embeddings=1), model.encode(paras1, show_progress_bar=1, normalize_embeddings=1).T)

sns.heatmap(cmat.T, cmap="Blues", cbar=True)

In [ ]:
# @title def plot_cmat
# @title  import SentenceTransformer
try:
  import sentence_transformers
except ModuleNotFoundError:
  !pip install -q sentence_transformers

from sentence_transformers import SentenceTransformer
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")

MODEL_ID1 = "sentence-transformers/distiluse-base-multilingual-cased-v2"
MODEL_ID2 = "sentence-transformers/distiluse-base-multilingual-cased-v1"  # dim512
MODEL_ID3 = "sentence-transformers/multi-qa-mpnet-base-dot-v1"  # .43G 8s dim768
MODEL_ID4 = "sentence-transformers/multi-qa-distilbert-cos-v1"  #
MODEL_ID5 = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"  # dim384
MODEL_ID6 = "paraphrase-multilingual-mpnet-base-v2"

def plot_cmat(MODEL_ID=None, paras1=None, paras2=None, bs=10):
# def plot_cmat1(MODEL_ID, txt1, txt2, bs=10):
  # _ = """
  if MODEL_ID is None:
    MODEL_ID = MODEL_ID1

  if paras1 is None:
    paras1 = en[:]
  else:
    display(len(paras1))
  if paras2 is None:
    paras2 = zh[:]
  else:
    display(len(paras2))
  # """

  # paras1 = txt1[:]
  # paras2 = txt2[:]

  display("******")
  display(f"{paras1=}, {paras2=}")

  display([len(paras1), len(paras2)])

  model = SentenceTransformer(MODEL_ID)
  display(f"{MODEL_ID=}, {model=}")

  v_en = model.encode(paras1, show_progress_bar=1, normalize_embeddings=1, batch_size=bs)
  v_zh = model.encode(paras2, show_progress_bar=1, normalize_embeddings=1, batch_size=bs)

  # display(v_en)
  # display(v_zh)

  display([v_en.shape, v_zh.shape])

  cmat = np.dot(v_zh, v_en.T)
  plot_cmat.cmat = cmat

  sns.heatmap(cmat, cmap="Blues", cbar=True)
  return cmat

# plot_cmat()

In [ ]:
# @title MODEL_ID1 "sentence-transformers/distiluse-base-multilingual-cased-v2"
_ = plot_cmat(MODEL_ID1)
_.shape

# 02:08 01:58

In [ ]:
_ = plot_cmat(MODEL_ID1, bs=16)

# bs32 02:15 01:54
# bs=16 02:10 01:51

In [ ]:
# @title MODEL_ID2 "sentence-transformers/distiluse-base-multilingual-cased-v1"
_ = plot_cmat(MODEL_ID2)
_.shape

# 02:12 01:53

In [ ]:
# @title MODEL_ID3 09:17 06:47 sentence-transformers/multi-qa-mpnet-base-dot-v1
_ = plot_cmat(MODEL_ID3)
_.shape

# 09:17 06:47

In [ ]:
# @title MODEL_ID4 sentence-transformers/multi-qa-distilbert-cos-v1
_ = plot_cmat(MODEL_ID4)
_.shape

# 04:28 03:22

In [ ]:
# @title **!! MODEL_ID5 01:21 01:01 sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2**
_ = plot_cmat(MODEL_ID5)
_.shape

# 01:21 01:01

In [ ]:
# @title MODEL_ID6
_ = plot_cmat(MODEL_ID6)
_.shape

#

In [ ]:
# @title "intfloat/multilingual-e5-large"

_ = plot_cmat("intfloat/multilingual-e5-large", en, zh)
_.shape

In [ ]:
# @title MODEL_ID1 sentence-transformers/multi-qa-mpnet-base-dot-v1
plot_cmat(MODEL_ID1)  # 1.51 s  9min 23 7min 15s

In [ ]:
plot_cmat()  # MODEL_ID2 2.08 s 4min 21s 3min 47s

In [ ]:
plot_cmat(MODEL_ID3)  # 8s, 2min 5s, 1min 50s

In [ ]:
plot_cmat(MODEL_ID4)  # 4s  1min 50s 1min

In [ ]:
# @title Nepali

txt1 = ["पनौतीका बाआमाले खानेकुरा खोज्न विहान सबेरै घर छाडेका थिए । छिमेकी प्राय: सबैले गाउँ छाडिसकेका थिए । गाउँमा चोर र डाँकाहरूको बिगबिगी सुरु भइसकेको थियो । पनौतीको घरमा पनि दिनदिनै केही अपरिचित मान्छेहरू आएर भित्र पस्थे अनि केही खानेकुरा नपाउँदा रिसाएर हिँड्थे ।",
        "तलको तालिकाले (चयन गरिएको) मोडेलहरूको सिंहावलोकन प्रदान गर्दछ। इम्बेडिङ वाक्यहरूमा तिनीहरूको गुणस्तर (कार्यसम्पादन वाक्य इम्बेडिङ) र इम्बेडिङ खोज प्रश्नहरू र अनुच्छेदहरू (कार्यसम्पादन सिमान्टिक खोज) व्यापक रूपमा मूल्याङ्कन गरिएको छ।"]

txt2 = ["帕诺蒂的父母每天一大早就出去寻找食物，几乎所有的邻居都已经匆匆离开了村子，留下了小偷和强盗在村中横行霸道。即便在帕诺蒂的家中，每天也有陌生人闯入，当得不到任何食物时，他们怒气冲天地离去。",
        "下表提供了（选定的）型号的概述。 它们的嵌入句子（性能句子嵌入）和嵌入搜索查询和段落（性能语义搜索）的质量已得到广泛评估。"]



In [ ]:
plot_cmat(MODEL_ID1, txt1, txt2)



In [ ]:
plot_cmat(MODEL_ID2, txt1, txt2)


In [ ]:
plot_cmat(MODEL_ID3, txt1, txt2)


In [ ]:
plot_cmat(MODEL_ID4, txt1, txt2)

In [ ]:
# sentence-transformers/distiluse-base-multilingual-cased-v2

plot_cmat("sentence-transformers/distiluse-base-multilingual-cased-v2", txt1, txt2)

In [ ]:
# @title MODEL_ID5
plot_cmat(MODEL_ID5, txt1, txt2)

In [ ]:
# @title "intfloat/multilingual-e5-large"

plot_cmat("intfloat/multilingual-e5-large", txt1, txt2)

In [ ]:
# @title  test_en.txt test_zh.txt
test_en = loadtext("en-de-zh-txt/test_en.txt", 1)
test_zh = loadtext("en-de-zh-txt/test_zh.txt", 1)

In [ ]:
# @title MODEL_ID5

plot_cmat(MODEL_ID5, test_en, test_zh)

In [ ]:
# @title MODEL_ID1

plot_cmat(MODEL_ID1, test_en, test_zh)

In [ ]:
# @title MODEL_ID2

plot_cmat(MODEL_ID2, test_en, test_zh)

In [ ]:
# @title pad
MODEL_ID1 = "sentence-transformers/multi-qa-mpnet-base-dot-v1"  # .43G 8s dim768
MODEL_ID2 = "sentence-transformers/multi-qa-distilbert-cos-v1"  # .25G dim768
# MODEL_ID3 = "sentence-transformers/distiluse-base-multilingual-cased-v1"  # dim512
# MODEL_ID4 = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"  # dim384

MODEL_ID = MODEL_ID2

paras1 = en[:]
paras2 = zh[:]

display(f"{len(paras1)=}, {len(paras2)=}")

%time model = SentenceTransformer(MODEL_ID)
display(f"{MODEL_ID=}, {model=}")

%time v_en = model.encode(paras1, show_progress_bar=1, normalize_embeddings=1)
%time v_zh = model.encode(paras2, show_progress_bar=1, normalize_embeddings=1)
%time cmat = np.dot(v_zh, v_en.T)

sns.heatmap(cmat.T, cmap="Blues", cbar=True)


In [ ]:
display(f"{len(paras1)=}, {len(paras2)=}")

%time model = SentenceTransformer(MODEL_ID)
%time v_en = model.encode(paras1, show_progress_bar=1, normalize_embeddings=1)

In [ ]:
# intfloat/e5-large-v2

plot_cmat("intfloat/e5-large-v2")


In [ ]:
# @title Test model-s-v2 mikeee/model_s_512v2
%%time

import joblib
from loguru import logger
from huggingface_hub import hf_hub_download

try:
  loc = hf_hub_download("mikeee/model_s_512", "model-s", local_dir=".")
  loc2 = hf_hub_download("mikeee/model_s_512v2", "model-s-v2", local_dir=".")
except Exception as exc:
  logger.error(exc)
  raise SystemExit(1) from exc



In [ ]:
%time model = joblib.load(loc)
%time model2 = joblib.load(loc2)

In [ ]:
import numpy as np
import seaborn as sns
sns.set(style="whitegrid")

encode = lambda vec: model.encode(vec, show_progress_bar=True, normalize_embeddings=True, batch_size=10)
encode2 = lambda vec: model2.encode(vec, show_progress_bar=True, normalize_embeddings=True, batch_size=10)

ax = sns.heatmap(np.dot(encode(test_zh), encode(test_en).T), cmap="Blues")

In [ ]:
ax2 = sns.heatmap(np.dot(encode2(test_zh), encode2(test_en).T), cmap="Blues")

In [ ]:
# model.encode?